In [6]:
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

import funcs.ArticlePage as ArticlePage
import funcs.AuthorPage as AuthorPage
import funcs.Common as Common
import networkx as nx
import importlib
import time


In [12]:
G = nx.gml.read_gml("../gmls/510_authors.gml", destringizer=int)
nx.info(G)

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_6392\2682943580.py:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G)


'Graph with 27180 nodes and 31355 edges'

In [3]:
firefox_service = Service()
web_driver = webdriver.Firefox(service = firefox_service)
url = "https://www.scopus.com//authid/detail.uri?authorId=10140494200"
web_driver.get(url)

url_prefix = "https://www.scopus.com/authid/detail.uri?authorId="

In [4]:
starting_ids = [
				# 7202074046, 	#Kessler, Ronald C. 		N.0001
				# 57203046809,	#Manson, Jo Ann E. 			N.0002
				# 57206520256,	#Colditz, Graham A.			N.0003
				# 7402409226,		#Langer, Robert Samuel M.	N.0004
				# 35606519600,	#Jackson, Jeremy B.C.		N.0005
				# 35396685700,	#Akira, Shizuo				N.0006
				# 36077704000,	#Vogelstein, Bert			N.0007
				# 35463345800,	#Grätzel, M.				N.0008
				# 36038688700,	#Hu, Frank B.				N.0009
				# 8841196600,		#Guyatt, Gordon Henry M.	N.0010
				# 35373488900,	#Cummings, Steven Ron		N.0100
				# 36013862000,	#Nakamura, Yusuke			N.0101
				# 7501533080,		#Wilson, Richard D.			N.0102
				# 35375959000,	#Stanley, H. Eugene			N.0103
				# 35492852400,	#Ajayan, Pulickel M.		N.0104
				# 7402574751,		#Reíter, Rüssel J.			N.0105
				# 55913409300,	#Tyler, Tom R.				N.1000
				# 7006099014,		#Mantzoros, Christos S.		N.1001
				# 34573429300,	#Sham, Pak Chung			N.1002
				# 7005510490,		#Ribas, Antoni				N.1003
				# 35227997700,	#Slutsky, Arthur S.			N.1004
				# 18038298200,	#Pałka, Marek				N.1005
				# 7103120387,		#Driessen, Arnold J.M.		N.4725
				# 7005903711,		#Hilsenbeck, Susan G.		N.4726
				# 26643614600,	#Kopecek, Jindrich I.		N.4727
				# 10840422600,	#Elde, Robert P.			N.4728
				# 7202938057,		#Foster, Tim J.				N.4729
				# 8502005900,		#Labbé, Ivo					N.4730
				# 7004586426,		#Kolodner, Paul
				# 35317963900,	#Kürths, Jürgen
				# 55958178100,	#Hertz, Paul
				# 16553015400,	#Neilsen, Joseph
				10140494200		#Rodrigues, Francisco Aparecido
				]

In [5]:
importlib.reload(ArticlePage)
_ = importlib.reload(AuthorPage)


In [ ]:
current_layer = set(starting_ids)
author_graph = nx.Graph()
explored_ids = set()
next_layer = set()

number_of_layers = 3
MAX_TRIES = 3


for layer in range(number_of_layers):
	progress_bar = tqdm(current_layer, desc=f"Layer {layer}")

	for author_id in progress_bar:
		progress_bar.set_description(f"Layer {layer} Author {author_id} Articles {0:3d}%", refresh=True)
		explored_ids.add(author_id) #Adiciona autor aos visitados
		
		attempts = 0
		success = False
		while not success and attempts < MAX_TRIES: #Loop de tentativas para lidar com erros de carregamento (404, 500, etc.)
			try:
				web_driver.get(url_prefix + str(author_id)) #Acessa página do autor
				time.sleep(4)
				AuthorPage.add_author(web_driver, author_id, author_graph) #Adiciona autor e dados ao grafo
				success = True
			except KeyboardInterrupt:
				raise
			except:
				attempts+=1

		if(not success): #Lida com falha de obter autor
			print(f"Failed Author for {author_id}")
			try:
				soup=BeautifulSoup(web_driver.page_source,'lxml')
				file=open(f"Fails\\Author_{author_id}.html", "w", encoding="utf-8")
				file.write(f"Layer: {layer}\tAuthor: {author_id}\n\n")
				file.write(str(soup))
				file.close()
			except KeyboardInterrupt:
				raise
			except:
				pass
			continue #Pula autor

		articles = AuthorPage.get_author_articles(web_driver, sleep_time=4) #Lista de artigos do pesquisador

		number_of_articles = len(articles)
		for current_article_index, article_url in enumerate(articles):
			progress_bar.set_description(f"Layer {layer} Author:{author_id} Articles:{(100*current_article_index)//number_of_articles}%", refresh=True)
			attempts = 0
			success = False
			while not success and attempts < MAX_TRIES: #Loop de tentativas para lidar com erros de carregamento (404, 500, etc.)
				try:
					web_driver.get(article_url) #Acessa página do Artigo
					time.sleep(0.8)
					ArticlePage.add_article(web_driver, explored_ids, next_layer, author_graph, author_id) #Adiciona conexões formadas pelo artigo
					success = True
				except KeyboardInterrupt:
					raise
				except:
					attempts+=1
				
				if(not success): #Lida com falha de obter artigo
					print(f"Failed Article for {author_id}")
					try:
						soup=BeautifulSoup(web_driver.page_source,'lxml')
						file=open(f"Fails\\Article_{author_id}.html", "w", encoding="utf-8")
						file.write(f"Layer: {layer}\tAuthor: {author_id}\n\n")
						file.write(str(soup))
						file.close()
					except KeyboardInterrupt:
						raise
					except:
						pass

	current_layer = next_layer.copy()
	next_layer = set()


Common.play_notification()

In [ ]:
name = f"../gmls/{number_of_layers}_layers.gml"
nx.write_gml(author_graph, str(name))


In [29]:
web_driver.quit()